imports

In [1]:
import pandas as pd

writer = pd.ExcelWriter('./results/data_of_corrects.xlsx', engine='xlsxwriter')

config

In [2]:
config_prefix = ['BR', 'BI', 'TR', 'TI']

config_datasheet = r'./results/preprocessed.xlsx'

import df

In [3]:
raw = pd.read_excel(config_datasheet)
df = pd.DataFrame(raw)
df = df.loc[df["Correctness"]==True]
df.to_excel(writer, sheet_name="corrects")

get names of algorithm

In [4]:
algo_name_array = [name for name in df["Algorithm"].unique()]

split dataframes 

In [5]:
df_cat_array = []
df_cat_array.append(df.loc[(df["ProgrammingStyle"]=="R") & (df["Comprehension"]=="BU")])
df_cat_array.append(df.loc[(df["ProgrammingStyle"]=="I") & (df["Comprehension"]=="BU")])
df_cat_array.append(df.loc[(df["ProgrammingStyle"]=="R") & (df["Comprehension"]=="TD")])
df_cat_array.append(df.loc[(df["ProgrammingStyle"]=="I") & (df["Comprehension"]=="TD")])

df_algo_cat_array = []
for algo_name in algo_name_array:
    tmp_df = []
    tmp_df.append(df.loc[(df["Algorithm"]==algo_name) & (df["ProgrammingStyle"]=="R") & (df["Comprehension"]=="BU")])
    tmp_df.append(df.loc[(df["Algorithm"]==algo_name) & (df["ProgrammingStyle"]=="I") & (df["Comprehension"]=="BU")])
    tmp_df.append(df.loc[(df["Algorithm"]==algo_name) & (df["ProgrammingStyle"]=="R") & (df["Comprehension"]=="TD")])
    tmp_df.append(df.loc[(df["Algorithm"]==algo_name) & (df["ProgrammingStyle"]=="I") & (df["Comprehension"]=="TD")])
    df_algo_cat_array.append(tmp_df)

create result

In [6]:
cols = ["Algorithm", "Comprehension",
        "R_ResponseTimeMean", "R_ResponseTimeDeviation",
        "I_ResponseTimeMean", "I_ResponseTimeDeviation"]

result_df = pd.DataFrame(columns=cols)

for algo_df_array in df_algo_cat_array:
    for i in range(2):
        rec =  algo_df_array[i*2+0]
        it =  algo_df_array[i*2+1]
        
        Algorithm = rec.iloc[0]["Algorithm"]
        Comprehension = "Top-Down"
        if i == 1:
            Comprehension = "Bottom-Up"
        
        R_ResponseTimeMean = int(rec["ResponseTime"].mean())
        R_ResponseTimeDeviation = int(rec["ResponseTime"].std())
        
        I_ResponseTimeMean = int(it["ResponseTime"].mean())
        I_ResponseTimeDeviation = int(it["ResponseTime"].std())
        
        data = [
            Algorithm, Comprehension,
            R_ResponseTimeMean, R_ResponseTimeDeviation,
            I_ResponseTimeMean, I_ResponseTimeDeviation,
        ]
        
        result_df = result_df.append(pd.DataFrame([data], columns=cols))

for i in range(2):
    rec =  df_cat_array[i*2+0]
    it =  df_cat_array[i*2+1]
    
    Algorithm = "Total"
    Comprehension = "Top-Down"
    if i == 1:
        Comprehension = "Bottom-Up"
    
    R_ResponseTimeMean = int(rec["ResponseTime"].mean())
    R_ResponseTimeDeviation = int(rec["ResponseTime"].std())
    
    I_ResponseTimeMean = int(it["ResponseTime"].mean())
    I_ResponseTimeDeviation = int(it["ResponseTime"].std())
    
    data = [
        Algorithm, Comprehension,
        R_ResponseTimeMean, R_ResponseTimeDeviation,
        I_ResponseTimeMean, I_ResponseTimeDeviation,
    ]
    
    result_df = result_df.append(pd.DataFrame([data], columns=cols))

print("Write total means to excel sheet")
result_df.to_excel(writer, sheet_name="corrects.info")
result_df

Write total means to excel sheet


,Algorithm,Comprehension,R_ResponseTimeMean,R_ResponseTimeDeviation,I_ResponseTimeMean,I_ResponseTimeDeviation
0,BinarySearch,Top-Down,441,201,283,261
0,BinarySearch,Bottom-Up,202,99,322,301
0,BubbleSort,Top-Down,260,156,261,218
0,BubbleSort,Bottom-Up,27,28,401,492
0,Factorial,Top-Down,132,251,139,105
0,Factorial,Bottom-Up,77,35,109,78
0,Fibonacci,Top-Down,180,143,240,144
0,Fibonacci,Bottom-Up,200,154,180,113
0,IntegerBinary,Top-Down,420,516,399,403
0,IntegerBinary,Bottom-Up,130,92,210,98


create means of means for total

In [7]:
cols = ["Algorithm", "Comprehension",
        "R_ResponseTimeMean", "R_ResponseTimeDeviation",
        "I_ResponseTimeMean", "I_ResponseTimeDeviation"]
means_per_algo = result_df[cols]
means_per_algo = means_per_algo.iloc[:-2,:]

df_top_down = means_per_algo.loc[means_per_algo["Comprehension"] == "Top-Down"]
df_bottom_up = means_per_algo.loc[means_per_algo["Comprehension"] == "Bottom-Up"]

In [8]:
means_bu_r = int(df_bottom_up["R_ResponseTimeMean"].mean())
means_bu_i = int(df_bottom_up["I_ResponseTimeMean"].mean())
means_td_r = int(df_top_down["R_ResponseTimeMean"].mean())
means_td_i = int(df_top_down["I_ResponseTimeMean"].mean())

stddev_bu_r = int(df_bottom_up["R_ResponseTimeMean"].std())
stddev_bu_i = int(df_bottom_up["I_ResponseTimeMean"].std())
stddev_td_r = int(df_top_down["R_ResponseTimeMean"].std())
stddev_td_i = int(df_top_down["I_ResponseTimeMean"].std())

df = pd.DataFrame([], columns=cols)
df = df.append(pd.DataFrame([["Total", "Top-Down", means_td_r, stddev_td_r, means_td_i, stddev_td_i]], columns=cols))
df = df.append(pd.DataFrame([["Total", "Bottom-Up", means_bu_r, stddev_bu_r, means_bu_i, stddev_bu_i]], columns=cols))
means_per_algo = means_per_algo.append(df)
means_per_algo.to_excel(writer, sheet_name="corrects.weighted_info")
display(means_per_algo)
writer.save()

,Algorithm,Comprehension,R_ResponseTimeMean,R_ResponseTimeDeviation,I_ResponseTimeMean,I_ResponseTimeDeviation
0,BinarySearch,Top-Down,441,201,283,261
0,BinarySearch,Bottom-Up,202,99,322,301
0,BubbleSort,Top-Down,260,156,261,218
0,BubbleSort,Bottom-Up,27,28,401,492
0,Factorial,Top-Down,132,251,139,105
0,Factorial,Bottom-Up,77,35,109,78
0,Fibonacci,Top-Down,180,143,240,144
0,Fibonacci,Bottom-Up,200,154,180,113
0,IntegerBinary,Top-Down,420,516,399,403
0,IntegerBinary,Bottom-Up,130,92,210,98
